In [1]:
# import packages
import pandas as pd # For reading and manipulating 2D data (like spreadsheets)
import numpy as np # For doing numerical calculations (literally NUMerical PYthon)
import matplotlib.pyplot as plt # For making graphs

from sklearn import datasets, linear_model

# Setting to show graphs below cells in colab
%matplotlib inline

In [2]:
temp = pd.read_csv(r"../data/Temp.csv")
sea = pd.read_csv(r"../data/Sea.csv")
sea["Delta Sea"] = sea["Sea"].diff()

joint = pd.merge(temp, sea, on='Year', how='inner')
joint = joint.dropna()

joint

,Year,Temp,Sea,Delta Sea
1,1881,-0.232246,-173.039065,7.197218
2,1882,-0.295530,-191.580733,-18.541668
3,1883,-0.346474,-190.455727,1.125005
4,1884,-0.492320,-166.597397,23.858330
5,1885,-0.471124,-168.555730,-1.958332
...,...,...,...,...
136,2016,0.932927,56.330711,4.151281
137,2017,0.845174,55.112391,-1.218321
138,2018,0.762654,52.083272,-3.029119
139,2019,0.891073,60.980739,8.897467


In [5]:
temp_array = joint["Temp"].to_numpy().reshape(-1, 1)
sea_array = joint["Delta Sea"].to_numpy().reshape(-1, 1)

regr = linear_model.LinearRegression(fit_intercept=True)
model = regr.fit(temp_array, sea_array)
print(f"The model [ y = {regr.coef_[0][0]} x + {regr.intercept_[0]}] has score " + 
      f"{regr.score(temp_array, sea_array)}.")


The model [ y = 2.125329390233103 x + 1.8330317620354901] has score 0.01909633397816135.
